# Export Hospital Level Dashboards

**Purpose:**

1. **Optionally authenticate** with `gcloud` (via user prompt).  
2. **Query BigQuery** for facility names (`faci_name` from `drg-viz.00_datasets.hci`).  
3. **(Optional) Save** these names to a local CSV for debugging.  
4. **Launch Selenium** (using `a_source_chrome_webdriver.py`) to open Power BI in **headless** mode,  
   select each facility in the slicer, and **export** a PDF to `./hospital-reports`.


In [ ]:
import os
import sys
import json

# Third-party imports
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError

# Local imports (the new approach)
project_root = os.path.expanduser("~/drg-hospital-dashboards")
if project_root not in sys.path:
    sys.path.append(project_root)

from src.functions import *

def main():
    """
    Full reload-from-scratch approach:
      1) We loop up to MAX_RETRIES.
      2) Each loop calls attempt_run().
      3) If attempt_run() returns True => success => break.
         Otherwise, increment WAIT_TIMES as desired and re-run.
      4) If we exceed MAX_RETRIES, we stop.
    """

    # Optionally, load WAIT_TIMES from JSON if it exists:
    if os.path.isfile(WAIT_TIMES_JSON):
        with open(WAIT_TIMES_JSON, "r") as f:
            loaded_times = json.load(f)
            WAIT_TIMES.update(loaded_times)
        print(f"Loaded WAIT_TIMES from {WAIT_TIMES_JSON}: {WAIT_TIMES}")

    attempt_count = 1
    while attempt_count <= MAX_RETRIES:
        print(f"\n=== Attempt #{attempt_count} (TO_DEBUG={TO_DEBUG}) with WAIT_TIMES={WAIT_TIMES} ===")
        success = attempt_run()
        if success:
            print("Run succeeded! Stopping.")
            break
        else:
            # For simplicity, let's increment a single wait time
            # or you can pick which step to increment. For example:
            WAIT_TIMES["dropdown_sleep"] += 1
            print(f"Run failed. Increased WAIT_TIMES['dropdown_sleep'] to {WAIT_TIMES['dropdown_sleep']}.")
            print("Will reload from scratch...\n")

        attempt_count += 1

    if attempt_count > MAX_RETRIES:
        print(f"Gave up after {MAX_RETRIES} attempts.")
        print(f"Final WAIT_TIMES: {WAIT_TIMES}")
    else:
        print(f"Success on attempt #{attempt_count}. Final WAIT_TIMES: {WAIT_TIMES}")

    # Save final WAIT_TIMES to JSON for next time
    with open(WAIT_TIMES_JSON, "w") as f:
        json.dump(WAIT_TIMES, f)
    print(f"Saved WAIT_TIMES to {WAIT_TIMES_JSON}.")


if __name__ == "__main__":
    main()
